In [3]:
import numpy as np
import random
import scipy.linalg
import gym
import sys
sys.path.append("../utility")
sys.path.append("../train")
from Utility import DerivativeLiftFunc, data_collecter,RBFLiftFunc

/home/stone/anaconda3/envs/mujoco/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [4]:
ENV = ["DampingPendulum","Pendulum-v1","CartPole-v1","MountainCarContinuous-v0","Franka","DoublePendulum"]

In [7]:
def evalKoopmanD(env_index):
    env_name = ENV[env_index]
    print(env_name)
    random.seed(1)
    np.random.seed(1)
    Data_collecter = data_collecter(env_name)
    Nstate = Data_collecter.Nstates
    udim = Data_collecter.udim
    Nrbf = 50
    LiftFunc = RBFLiftFunc(env_name,Nstate,udim,Nrbf,Data_collecter.observation_space)
    NKoopman = LiftFunc.NKoopman
    Samples = 5000
    Steps = 30
    train_data = Data_collecter.collect_koopman_data(20000,1)
    X_lift = LiftFunc.Psi_su(train_data[0,:,udim:],train_data[0,:,:udim])
    Y_lift = LiftFunc.Psi_su(train_data[1,:,udim:],train_data[0,:,:udim])
    XX = np.dot(X_lift.T,X_lift)
    XY = np.dot(X_lift.T,Y_lift)
    Kd = np.dot(scipy.linalg.pinv2(XX),XY).T  
    np.savez("Prediction_Results/"+"Kd_"+env_name+"_KoopmanRBF"+".npz",Kd=Kd,Center=LiftFunc.center)  
    times = 4 
    max_loss_all = np.zeros((times,Steps))
    mean_loss_all = np.zeros((times,Steps))
    for t in range(times):
        test_data = Data_collecter.collect_koopman_data(Samples,Steps)
        X_current = test_data[0,:,:]
        X_current_lift = np.zeros((Samples,NKoopman+udim))
        maxloss_list = []
        meanloss_list = []
        for i in range(Samples):
            X_current_lift[i] = LiftFunc.Psi_su(X_current[i,udim:],X_current[i,:udim])
        for i in range(Steps):
            X_current_lift = np.dot(X_current_lift,Kd.T)
            X_current_lift[:,NKoopman:] = test_data[i+1,:,:udim]
            Err = X_current_lift[:,:Nstate] - test_data[i+1,:,udim:]
            maxloss = np.mean(np.max(abs(Err),axis=0))
            meanloss = np.mean(np.mean(abs(Err),axis=0))
            maxloss_list.append(maxloss)
            meanloss_list.append(meanloss)
        max_loss_all[t] = np.array(maxloss_list).reshape(-1)
        mean_loss_all[t] = np.array(meanloss_list).reshape(-1)
    max_mean = np.mean(max_loss_all,axis=0)
    max_std = np.std(max_loss_all,axis=0)
    mean_mean =  np.mean(mean_loss_all,axis=0)
    mean_std =  np.std(mean_loss_all,axis=0)  
    np.save("Prediction_Results/"+env_name+"_KoopmanRBF"+".npy",np.array([max_mean,max_std,mean_mean,mean_std]))
    return max_mean,max_std,mean_mean,mean_std 

In [9]:
for j in range(6):
    evalKoopmanD(j)

DampingPendulum


/home/stone/anaconda3/envs/mujoco/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: scipy.linalg.pinv2 is deprecated since SciPy 1.7.0, use scipy.linalg.pinv instead


Pendulum-v1
CartPole-v1
MountainCarContinuous-v0
Reacher-v2
Franka
